In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

from dotenv import load_dotenv
import sys
import os
import datetime
# Добавляем корневую директорию проекта в sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

from consultant.tools.basic import normalize_path

load_dotenv()

ENVIRONMENT = os.getenv('ENVIRONMENT')
HOME_DIR = os.getenv('HOME_DIR')
FIXTURE_DIR = os.getenv('FIXTURE_DIR')

spark = (SparkSession.builder
        .appName("Clients_etl")
        .getOrCreate())

spark

KeyboardInterrupt: 

In [2]:
path = normalize_path(*[HOME_DIR, FIXTURE_DIR, 'clients.csv'])
clients = spark.read.csv(path, header=True, inferSchema=True)


clients.show(20, False)
clients.printSchema()

+---------+-------------------+----------------------------+----------------------+--------------------------------------------------------+
|client_id|client_name        |client_email                |client_phone          |client_address                                          |
+---------+-------------------+----------------------------+----------------------+--------------------------------------------------------+
|1        |Douglas Mann       |malexander@example.net      |7887828319            |447 Sanders Forge Apt. 868 Port Lisafort, MT 13535      |
|2        |Linda Willis       |sherryriddle@example.org    |648.448.5400x89273    |4186 Richard Turnpike Cynthiaton, GA 06049              |
|3        |Mr. Samuel Gonzalez|adam37@example.com          |+1-727-264-1902x10293 |5333 Bradley Corner Apt. 956 Port Michael, UT 04993     |
|4        |Elijah Duke        |alexis42@example.com        |+1-913-911-7052x28833 |6919 Strong Square South Catherine, OR 42933            |
|5        |An

**Cleaning clients name from degrees and statuses**


In [3]:
from consultant.patterns.names import TITLES_REGEX, DEGREES_REGEX


filtered_by_name = clients \
    .withColumn(
        'client_name',
        when(
            size(split(col('client_name'), r"\s+")) >= 3,
            regexp_replace(
            col('client_name'),
            rf"(^({TITLES_REGEX})\s*)|(\s+({DEGREES_REGEX})$)",
            "")
        ) \
        .otherwise(col('client_name'))
    )

filtered_by_name.show(100)


+---------+--------------------+--------------------+--------------------+--------------------+
|client_id|         client_name|        client_email|        client_phone|      client_address|
+---------+--------------------+--------------------+--------------------+--------------------+
|        1|        Douglas Mann|malexander@exampl...|          7887828319|447 Sanders Forge...|
|        2|        Linda Willis|sherryriddle@exam...|  648.448.5400x89273|4186 Richard Turn...|
|        3|     Samuel Gonzalez|  adam37@example.com|+1-727-264-1902x1...|5333 Bradley Corn...|
|        4|         Elijah Duke|alexis42@example.com|+1-913-911-7052x2...|6919 Strong Squar...|
|        5|      Angela Michael|mistybaker@exampl...|          5118303852|9280 Natalie Spri...|
|        6|    Jennifer Bullock|hmaxwell@example.net|   (747)809-1366x312|95451 Matthew Pla...|
|        7|       Barbara Davis|kellyheath@exampl...|     +1-688-726-1099|39792 Torres Stre...|
|        8|      Jonathan Smith|michael0

**Parse and process phonenums**

In [4]:
cleaned_phones = filtered_by_name \
    .withColumn('client_phone',
                regexp_replace(
                    col('client_phone'),
                    rf"[^0-9x]",
                    ""
                )) \
    .withColumn('client_phone', concat(lit('+') ,col('client_phone')))
    

cleaned_phones.select('client_address').show(100, truncate=False)

+----------------------------------------------------------+
|client_address                                            |
+----------------------------------------------------------+
|447 Sanders Forge Apt. 868 Port Lisafort, MT 13535        |
|4186 Richard Turnpike Cynthiaton, GA 06049                |
|5333 Bradley Corner Apt. 956 Port Michael, UT 04993       |
|6919 Strong Square South Catherine, OR 42933              |
|9280 Natalie Spring Blakechester, MO 34228                |
|95451 Matthew Place Suite 985 East Monicaport, GU 05199   |
|39792 Torres Streets Suite 918 South Paulville, MD 10103  |
|4238 Moran Points Suite 072 New Wanda, GA 32488           |
|646 Laura Groves Apt. 156 West Joseph, RI 28565           |
|271 Garza Gardens Johnsontown, MA 68617                   |
|01703 Garcia Corner Apt. 423 Kentshire, AZ 94260          |
|39494 Alison Knoll Hughesfort, WA 01765                   |
|Unit 3904 Box 9417 DPO AE 98817                           |
|8921 Gonzalez Squares S

**Address clearing**

In [5]:
cleaned_address = cleaned_phones \
    .withColumn(
        'client_address',
        regexp_replace(
            col('client_address'),
            rf"[\,]",
            ""
        )
    )
cleaned_address.show(100, truncate=False)

+---------+--------------------+-----------------------------+--------------------+---------------------------------------------------------+
|client_id|client_name         |client_email                 |client_phone        |client_address                                           |
+---------+--------------------+-----------------------------+--------------------+---------------------------------------------------------+
|1        |Douglas Mann        |malexander@example.net       |+7887828319         |447 Sanders Forge Apt. 868 Port Lisafort MT 13535        |
|2        |Linda Willis        |sherryriddle@example.org     |+6484485400x89273   |4186 Richard Turnpike Cynthiaton GA 06049                |
|3        |Samuel Gonzalez     |adam37@example.com           |+17272641902x10293  |5333 Bradley Corner Apt. 956 Port Michael UT 04993       |
|4        |Elijah Duke         |alexis42@example.com         |+19139117052x28833  |6919 Strong Square South Catherine OR 42933              |
|5    

Deduplicating

In [6]:
prepared_clients = cleaned_address \
    .dropDuplicates() \
    .na.drop('all')
prepared_clients.select(count(expr("*"))).show(1)
prepared_clients.show(100, truncate=False)

+--------+
|count(1)|
+--------+
|   10000|
+--------+

+---------+-------------------+--------------------------------+--------------------+----------------------------------------------------------+
|client_id|client_name        |client_email                    |client_phone        |client_address                                            |
+---------+-------------------+--------------------------------+--------------------+----------------------------------------------------------+
|178      |Joel Boone         |christopher73@example.org       |+0019966139222      |0866 Danielle Stravenue Port Charlesview NM 80251         |
|1090     |Juan Phillips      |edward80@example.net            |+18205722750x996    |65210 Matthew Spurs North Samantha AL 51090               |
|1099     |Melissa Griffin    |michael76@example.com           |+0015868887738x21966|3449 Mendoza Ports Lake John VA 47513                     |
|1301     |Matthew Dougherty  |joseph28@example.net            |+469509372

In [ ]:
import importlib
import consultant.tools.basic as module

importlib.reload(module)

from consultant.tools.basic import get_hdfs_url

hdfs_url = get_hdfs_url('clients')
print(hdfs_url)


prepared_clients \
    .write.mode('overwrite') \
    .csv(hdfs_url)

hdfs://172.17.0.23//user/b.kustov/prepared_data/clients.csv


In [8]:

spark.stop()